<img src="tmva_logo.gif" height="20%" width="20%">


# Regression Example

Example of TMVA for Regression

### Create Factory class

Create a factory class for the regression. It is important to specify that the AnalysisType is in this case **Regression**

In [ ]:
TMVA::Tools::Instance();

auto inputFile = TFile::Open("inputdata_regression.root");
auto outputFile = TFile::Open("TMVA_RegressionOutput.root", "RECREATE");

TMVA::Factory factory("TMVARegression", outputFile,
                      "!V:!Silent:Color:!DrawProgressBar:AnalysisType=Regression" ); 

### Create DataLoader class

Cretae the DataLoader class and define the input features and the target for the regression.

In [ ]:
TMVA::DataLoader loader("dataset"); 

// Add the feature variables, names reference branches in inputFile ttree
loader.AddVariable("var1");
loader.AddVariable("var2");
loader.AddVariable("var3");
loader.AddVariable("var4");
loader.AddVariable("var5 := var1-var3"); // create new features
loader.AddVariable("var6 := var1+var2");

loader.AddTarget( "target := var2+var3" ); // define the target for the regression


### Setup Dataset
Link dataloader to dataset and prepare the data

In [ ]:
TTree *tree;
inputFile->GetObject("Sig", tree);

TCut mycuts = ""; // e.g. TCut mycuts = "abs(var1)<0.5";

loader.AddRegressionTree(tree, 1.0);   // link the TTree to the loader, weight for each event  = 1
loader.PrepareTrainingAndTestTree(mycuts,
                                   "nTrain_Regression=1000:nTest_Regression=1000:SplitMode=Random:NormMode=NumEvents:!V" );

## Book the regression method

Book the method for regression. Here we choose the Boosted Decision Tree model. You have to use gradient boosted trees for regression, hence the BDTG and BoostType=Grad. 

Define the hyperparameters: ntrees, boosttype, shrinkage, and the depth. Also define the loss function you want to use: 'AbsoluteDeviation', 'Huber', or 'LeastSquares'. nCuts determines how finely to look at each feature. Larger values take more time, but you may get more accurate results.

In [ ]:
// Boosted Decision Trees 
factory.BookMethod(&loader,TMVA::Types::kBDT, "BDTG_HU",
                   TString("!H:!V:NTrees=64::BoostType=Grad:Shrinkage=0.3:nCuts=20:MaxDepth=4:")+
                   TString("RegressionLossFunctionBDTG=Huber"));

In [ ]:
// Boosted Decision Trees 
factory.BookMethod(&loader,TMVA::Types::kBDT, "BDTG_LS",
                   TString("!H:!V:NTrees=64::BoostType=Grad:Shrinkage=0.3:nCuts=20:MaxDepth=4:")+
                   TString("RegressionLossFunctionBDTG=LeastSquares"));

# Train Method

In [ ]:
factory.TrainAllMethods();

# Test and Evaluate the Model

In [ ]:
factory.TestAllMethods();    

In [ ]:
factory.EvaluateAllMethods();

## Gather and Plot the Results
Let's plot the residuals for the BDTG predictions. First, close the output file so that it saves to disk and we can open it without issue. Then get the results on the test set. Finally, plot the residuals.

In [ ]:
//%jsroot on
outputFile->Close();
auto resultsFile = TFile::Open("TMVA_RegressionOutput.root");
auto resultsTree = (TTree*) resultsFile->Get("dataset/TestTree"); 
// BDTG-AD is the predicted value, target is the true value
auto h1 = new TH1D("h1","BDTG-target",100,1,0);
resultsTree->Draw("BDTG_HU-target >> h1"); 
auto h2 = (TH1*) h1->Clone("h2");
resultsTree->Draw("BDTG_LS-target >> h2"); 
h1->SetLineColor(kBlue); h1->Draw();
h2->SetLineColor(kRed); h2->Draw("SAME");
gPad->Draw();